### Create features 

In [33]:
import pandas as pd
import pickle
import operator

In [2]:
from IPython.display import display
pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.set_option('display.float_format', lambda x: '%.0f' % x)

In [3]:
with open('../data_preprocessing/data/svo_df.pkl', 'rb') as f:
    df = pickle.load(f)

In [53]:
def svo_counts(df, label_col):
    svo_counts = {
        'subject': {'total': {}},
        'verb': {'total': {}},
        'object': {'total': {}}
    }
    
    labels = df[label_col].unique()
    for label in labels:
        for svo_key in svo_counts.keys():
            svo_counts[svo_key][label] = {}

        # all svos for one label
        label_svos = df[df[label_col] == label].reset_index()
        for i in range(label_svos.shape[0]):
            user_svo_list = label_svos.loc[i,'svos']
            for tweet_svo_list in user_svo_list:
                for svo in tweet_svo_list:
                    for j in range(3):
                        word = svo[j]
                        if j == 0:
                            svo_key = 'subject'
                        elif j == 1:
                            svo_key = 'verb'
                        elif j == 2:
                            svo_key = 'object'
                        
                        # build dictionary
                        if word in svo_counts[svo_key]['total']:
                            svo_counts[svo_key]['total'][word] += 1
                            if word in svo_counts[svo_key][label]:
                                svo_counts[svo_key][label][word] += 1
                            else:
                                svo_counts[svo_key][label][word] = 1
                        else:
                            svo_counts[svo_key]['total'][word] = 1
                            svo_counts[svo_key][label][word] = 1                    
    return svo_counts


In [54]:
c = svo_counts(df, 'label')

In [55]:
def svo_lists(svo_counts, labels, top_k):
    svo_list_dict = {}
    svo_only_label_dict = {}
    
    for svo_key in svo_counts.keys():
        # find top k words per label
        svo_list_dict[svo_key] = {}
        svo_only_label_dict[svo_key] = {}
        for label in labels:            
            sorted_svo_list = sorted(svo_counts[svo_key][label].items(), key=operator.itemgetter(1))
            sorted_svo_list.reverse()
            svo_list_dict[svo_key][label] = sorted_svo_list[: top_k]
        
        # find unique and common words between labels (only works for 2 labels!)
        label1_words = [x[0] for x in svo_list_dict[svo_key][labels[0]]]
        label2_words = [x[0] for x in svo_list_dict[svo_key][labels[1]]]
        label1_unique = [word for word in label1_words if word not in label2_words]
        label2_unique = [word for word in label2_words if word not in label1_words]
        common = [word for word in label1_words if word in label2_words]
        svo_only_label_dict[svo_key][labels[0]] = label1_unique
        svo_only_label_dict[svo_key][labels[1]] = label2_unique
        svo_only_label_dict[svo_key]['common in top ' + str(top_k)] = common
            
    return svo_list_dict, svo_only_label_dict
        

In [60]:
svo_list_dict, svo_only_label_dict = svo_lists(c, ['d','r'], 20)

In [62]:
def create_sentiment_features(df, svo_only_label_dict, window):
    
    # subject
    
    

In [59]:
d2

{'object': {'common in top 20': ['',
   'https',
   'it',
   'convention',
   'amp',
   'you',
   'trump',
   'speech',
   'us',
   'president',
   'me',
   'people',
   'her'],
  'd': ['demsinphilly',
   'clinton',
   'dnc',
   'demconvention',
   'night',
   'them',
   'party'],
  'r': ['rncincle',
   'gopconvention',
   'america',
   'time',
   'him',
   'graph',
   'http…']},
 'subject': {'common in top 20': ['',
   'we',
   'you',
   'it',
   'they',
   'she',
   'he',
   'https',
   'trump',
   'people',
   'convention',
   '’'],
  'd': ['dnc',
   'clinton',
   'obama',
   'i',
   'hillaryclinton',
   'demsinphilly',
   'hillary',
   'supporters'],
  'r': ['rncincle',
   'cruz',
   'gop',
   'melania',
   'speech',
   'america',
   'realdonaldtrump',
   'republicans']},
 'verb': {'common in top 20': ['be ',
   '',
   "'s ",
   'have ',
   'say ',
   'get ',
   'do ',
   'make ',
   'https ',
   'want ',
   'speak ',
   'go ',
   'see ',
   'know ',
   'think ',
   'watch ',
   'n

In [6]:
# define parameters for feature generation
proto_word_args = {
    'text_col': 'full_text_agg', 
    'user_id': 'user_id', 
    'tok_type': 'clean', 
    'isalpha': True,
    'top_k': 100,
    'word_count_thresh': 5
}

hashtag_args = {
    'text_col': 'hashtags_agg', 
    'user_id': 'user_id',
    'top_k': 50,
    'ht_count_thresh': 3
}

topic_model_args = {
    'text_col': 'clean_text_agg',
    'user_id': 'user_id',
    'stops': nltk.corpus.stopwords.words('english') + ['rt'],
    'stemmer': nltk.stem.snowball.SnowballStemmer('english'), 
    'lemmer': None
}

topic_model_params = {
    'num_topic': 20, 
    'max_df': 0.5, 
    'min_df': 1, 
    'max_feature': 1000, 
    'alpha': 0.1, 
    'eta': 0.1,  
    'serialized': None 
}

In [7]:
X_train_ft, X_test_ft, y_train, y_test = feature.featurize(aggregated, 'label', proto_word_args, hashtag_args, topic_model_args, topic_model_params, 0.2, random_state=None, topic_words=False)

C:\Users\Sun-joo Lee\Git_Projects\Twitter\twitter_networks\feature.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  X_train[label] = y_train
C:\Users\Sun-joo Lee\Git_Projects\Twitter\twitter_networks\topics.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['tokenized_text'] = df[text_col].apply(lambda x: nltk.word_tokenize(x))
C:\Users\Sun-joo Lee\Git_Projects\Twitter\twitter_networks\topics.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = 

### Classifiers

In [31]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, BaggingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

In [35]:
clfs = {'Random Forest': RandomForestClassifier(n_estimators=100, n_jobs=-1),
        'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, learning_rate=0.05, subsample=0.5, max_depth=5),
        'AdaBoost': AdaBoostClassifier(DecisionTreeClassifier(max_depth=1), algorithm="SAMME", n_estimators=100),
        'Bagging, DT': BaggingClassifier(DecisionTreeClassifier(max_depth=1), max_samples=0.5, max_features=0.5),
        'Naive Bayes': GaussianNB(),        
        'Logistic Reg': LogisticRegression(penalty='l1', C=1e5),
        'SVM': SVC(kernel='rbf', probability=True, random_state=0),
        'Decision Tree': DecisionTreeClassifier()
            }

In [41]:
def basic_loop(clfs, X_train, y_train, X_test, y_test):
    for key, clf in clfs.items():
        print(key)
        start_time = time.time()
        clf_fit = clf.fit(X_train, y_train)
        print('Score: ', clf_fit.score(X_test, y_test))
        print("--- %s minutes ---" % round((time.time() - start_time)/60, 2))
        print()

In [42]:
basic_loop(clfs, X_train_ft, y_train, X_test_ft, y_test)

Random Forest
Score:  0.8782346685572492
--- 0.08 minutes ---

Gradient Boosting
Score:  0.8566111308046792
--- 0.69 minutes ---

AdaBoost
Score:  0.84774902516838
--- 0.27 minutes ---

Bagging, DT
Score:  0.848103509393832
--- 0.02 minutes ---

Naive Bayes
Score:  0.5233959588798298
--- 0.01 minutes ---

Logistic Reg
Score:  0.8459766040411202
--- 2.78 minutes ---

SVM
Score:  0.5235732009925558
--- 92.96 minutes ---

Decision Tree
Score:  0.8186813186813187
--- 0.02 minutes ---



In [23]:
clf = Perceptron().fit(X_train_ft, y_train)
clf.score(X_test_ft, y_test) 

0.47660404112017013

In [26]:
clf =SVC().fit(X_train_ft, y_train)
clf.score(X_test_ft, y_test) 

0.5235732009925558

In [2]:
a = pd.read_csv("X_train_ft.csv")

In [8]:
len(b.columns)

323